# Imports

In [ ]:
import numpy as np
import xarray as xr
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Polygon
from matplotlib import colors as mat_colors
import mpl_toolkits.axisartist as axisartist
from mpl_toolkits.axes_grid1 import Size, Divider

# Define Functions

## Performance measurements

In [ ]:
def BIAS(a1, a2):
    return (a1 - a2).mean().item()


def RMSE(a1, a2):
    return np.sqrt(((a1 - a2)**2).mean()).item()


def DIFF(a1, a2):
    return np.max(np.abs(a1 - a2)).item()

## help function for heatmap axis

In [ ]:
def setup_axes(fig, rect):
    ax = axisartist.Subplot(fig, rect)
    fig.add_subplot(ax)

    return ax

## heatmap

In [ ]:
def heatmap(datasets, # first_dataset, second_dataset,
            opti_var,
            annotation=None,
            annotation_x_position=0,
            annotation_y_position=1,
            fig=None, ax=None,
            cmap='vlag',
            cmap_levels=None,
            grid_color='grey',
            grid_linewidth=1.5,
            presentation=False,
            labels_pad=-360,
            xlim=None, # here use it do define max Diff sfc_h
            nr_of_iterations=None):
    if not ax:
        ax = plt.gca()
    
    if not fig:
        fig = plt.gcf()

    if all(dataset is None for dataset in datasets):
        raise ValueError('All datasets are None!')

    # define variables for plotting
    guess_opti_var = []
    first_guess_diff = []
    true_opti_var = []
    BIAS_opti_var = []
    RMSE_opti_var = []
    DIFF_opti_var = []
    fct_opti_var = []
    times = []
    maxiters = []
    BIAS_sfc = []
    RMSE_sfc = []
    DIFF_sfc = []
    BIAS_w = []
    RMSE_w = []
    DIFF_w = []
    BIAS_fg = []
    RMSE_fg = []
    DIFF_fg = []
    BIAS_sfc_fg = []
    RMSE_sfc_fg = []
    DIFF_sfc_fg = []
    array_length = 0
    check_first_guess = None
    check_true_opti_var = None

    # create data and label variables
    for dataset in datasets:
        # check if the current dataset contains data or if the data was not available
        if dataset is None:
            guess_opti_var.append(None)
            first_guess_diff.append(None)
            true_opti_var.append(None)
            BIAS_opti_var.append(None)
            RMSE_opti_var.append(None)
            DIFF_opti_var.append(None)
            fct_opti_var.append(None)
            times.append(None)
            maxiters.append(None)
            BIAS_sfc.append(None)
            RMSE_sfc.append(None)
            DIFF_sfc.append(None)
            BIAS_w.append(None)
            RMSE_w.append(None)
            DIFF_w.append(None)
        elif type(dataset) != xr.core.dataset.Dataset:  # if no minimisation possible
            guess_opti_var.append('no_minimisation')
            first_guess_diff.append(None)
            true_opti_var.append(None)
            BIAS_opti_var.append(None)
            RMSE_opti_var.append(None)
            DIFF_opti_var.append(None)
            fct_opti_var.append(None)
            times.append(None)
            maxiters.append(None)
            BIAS_sfc.append(None)
            RMSE_sfc.append(None)
            DIFF_sfc.append(None)
            BIAS_w.append(None)
            RMSE_w.append(None)
            DIFF_w.append(None)
        else:
            # find index corresponding to max time
            max_index = len(dataset['computing_time'].values) - 1

            if nr_of_iterations is not None:
                max_index = nr_of_iterations - 1
            elif xlim is not None:
                # calculate all max diff surface_h
                all_DIFF_sfc_h = np.array(
                    [DIFF(dataset.true_surface_h.data,
                          dataset.surface_h.data[i-1])
                     for i in dataset.coords['nr_of_iteration'].data])
                # only consider as many points until max DIFF is smaller xlim
                if all_DIFF_sfc_h[-1] < xlim:
                    max_index = np.argmax(all_DIFF_sfc_h < xlim)

            if opti_var == 'bed_h':
                guess_opti_var.append((dataset.guessed_bed_h[max_index] - dataset.true_bed_h).values)
                first_guess_diff.append((dataset.first_guessed_bed_h - dataset.true_bed_h).values)
                true_opti_var.append(dataset.true_bed_h.values)
                BIAS_opti_var.append(BIAS(dataset.guessed_bed_h[max_index], dataset.true_bed_h))
                RMSE_opti_var.append(RMSE(dataset.guessed_bed_h[max_index], dataset.true_bed_h))
                DIFF_opti_var.append(DIFF(dataset.guessed_bed_h[max_index], dataset.true_bed_h))
                if check_first_guess is None:
                    BIAS_fg = BIAS(dataset.first_guessed_bed_h, dataset.true_bed_h)
                    RMSE_fg = RMSE(dataset.first_guessed_bed_h, dataset.true_bed_h)
                    DIFF_fg = DIFF(dataset.first_guessed_bed_h, dataset.true_bed_h)
            elif opti_var == 'bed_shape':
                guess_opti_var.append((dataset.guessed_bed_shape[-1] - dataset.true_bed_shape).values)
                first_guess_diff.append((dataset.first_guessed_bed_shape - dataset.true_bed_shape).values)
                true_opti_var.append(dataset.true_bed_shape.values)
                BIAS_opti_var.append(BIAS(dataset.guessed_bed_shape[max_index], dataset.true_bed_shape))
                RMSE_opti_var.append(RMSE(dataset.guessed_bed_shape[max_index], dataset.true_bed_shape))
                DIFF_opti_var.append(DIFF(dataset.guessed_bed_shape[max_index], dataset.true_bed_shape))
                if check_first_guess is None:
                    BIAS_fg = BIAS(dataset.first_guessed_bed_shape, dataset.true_bed_shape)
                    RMSE_fg = RMSE(dataset.first_guessed_bed_shape, dataset.true_bed_shape)
                    DIFF_fg = DIFF(dataset.first_guessed_bed_shape, dataset.true_bed_shape)
            elif opti_var == 'w0':
                guess_opti_var.append((dataset.guessed_w0[-1] - dataset.true_w0).values)
                first_guess_diff.append((dataset.first_guessed_w0 - dataset.true_w0).values)
                true_opti_var.append(dataset.true_w0.values)
                BIAS_opti_var.append(BIAS(dataset.guessed_w0[max_index], dataset.true_w0))
                RMSE_opti_var.append(RMSE(dataset.guessed_w0[max_index], dataset.true_w0))
                DIFF_opti_var.append(DIFF(dataset.guessed_w0[max_index], dataset.true_w0))
                if check_first_guess is None:
                    BIAS_fg = BIAS(dataset.first_guessed_w0, dataset.true_w0)
                    RMSE_fg = RMSE(dataset.first_guessed_w0, dataset.true_w0)
                    DIFF_fg = DIFF(dataset.first_guessed_w0, dataset.true_w0)
            else:
                raise ValueError('Unknown opti var!')
                
            fct_opti_var.append(dataset.function_calls[max_index].values)
            times.append(dataset.computing_time[max_index].values)
            maxiters.append(dataset.attrs['maxiter_reached'])
            BIAS_sfc.append(BIAS(dataset.surface_h[max_index], dataset.true_surface_h))
            RMSE_sfc.append(RMSE(dataset.surface_h[max_index], dataset.true_surface_h))
            DIFF_sfc.append(DIFF(dataset.surface_h[max_index], dataset.true_surface_h))
            BIAS_w.append(BIAS(dataset.widths[max_index], dataset.true_widths))
            RMSE_w.append(RMSE(dataset.widths[max_index], dataset.true_widths))
            DIFF_w.append(DIFF(dataset.widths[max_index], dataset.true_widths))

            # determine array length for empty lines
            if array_length == 0:
                array_length = dataset.points_with_ice[-1].values + 1
            # check that the arrays have the same number of points with ice
            elif array_length != dataset.points_with_ice[-1].values + 1:
                raise ValueError('Not the same lentgth of points with ice!!!')

            # check if all experiments start with the same true values and first guess
            # in the first round save values
            if check_first_guess is None:
                check_first_guess = first_guess_diff[-1]
                check_true_opti_var = true_opti_var[-1]
                
                # not implemented yet
                BIAS_sfc_fg = BIAS(dataset.first_guess_surface_h, dataset.true_surface_h)
                RMSE_sfc_fg = RMSE(dataset.first_guess_surface_h, dataset.true_surface_h)
                DIFF_sfc_fg = DIFF(dataset.first_guess_surface_h, dataset.true_surface_h)
                BIAS_w_fg = BIAS(dataset.first_guess_widths, dataset.true_widths)
                RMSE_w_fg = RMSE(dataset.first_guess_widths, dataset.true_widths)
                DIFF_w_fg = DIFF(dataset.first_guess_widths, dataset.true_widths)

            # after first round compare all values to first ones to make sure comparing the same start conditions
            else:
                if np.sum(check_true_opti_var - true_opti_var[-1]) != 0:
                    raise ValueError('Not the same true control variable!!!')
                if np.sum(check_first_guess - first_guess_diff[-1]) != 0:
                    raise ValueError('Not the same first guess!!!')

    # create variables for ploting (data and y label)
    data = []
    y_labels = []

    # first add heading
    data.append(np.empty((array_length)) * np.nan)
    if not presentation:
        if opti_var == 'bed_h':
            y_labels.append(r'    RMSE_b,  DIFF_b,  RMSE_s,  DIFF_s,  fct,  $T_{cpu}$')
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append(r'   RMSE_Ps, DIFF_Ps,  RMSE_w,  DIFF_w,  fct,  $T_{cpu}$')
        else:
            raise ValueError('Unknown opti_var !')
        y_label_variable_format = '{:7.2f}, {: 7.2f}, {:7.2f}, {:7.2f}'
    else:
        if opti_var == 'bed_h':
            y_labels.append('   DIFF_b, fct, t')
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append('     DIFF  DIFF_w  fct  time')
        else:
            raise ValueError('Unknown opti_var !')
        y_label_variable_format = '{: 6.2f}' #', {:6.2f}'

    if not presentation:
        # add first guess
        data.append(check_first_guess)
        if opti_var == 'bed_h':
            y_labels.append(('fg:' + y_label_variable_format).format(RMSE_fg, DIFF_fg,
                                                                     RMSE_sfc_fg, DIFF_sfc_fg))
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append(('fg:' + y_label_variable_format).format(RMSE_fg, DIFF_fg,
                                                                     RMSE_w_fg, DIFF_w_fg))
        else:
            raise ValueError('Unknown opti_var !')
    else:
        # add first guess
        data.append(check_first_guess)
        if opti_var == 'bed_h':
            y_labels.append(('fg:' + y_label_variable_format).format(DIFF_fg))
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append(('fg:' + y_label_variable_format).format(DIFF_fg, DIFF_w_fg))
        else:
            raise ValueError('Unknown opti_var !')
    
    # add two format placeholders for fct_calls and time
    y_label_variable_format += ', {:4d}, {:4.0f}s'

    # add all other data with empty line for None datasets
    for i, guess in enumerate(guess_opti_var):
        if guess is None:
            data.append(np.empty((array_length)) * np.nan)
            if i < 9:
                y_labels.append((' ' + chr(65+i) + ':    NO DATAFILE FOUND'))
            else:
                y_labels.append((' ' + chr(65+i) + ':    NO DATAFILE FOUND'))
        elif type(guess) is str:
            data.append(np.empty((array_length)) * np.nan)
            if i < 9:
                y_labels.append((' ' + chr(65+i) + ':    NO Minimisation Possible'))
            else:
                y_labels.append((' ' + chr(65+i) + ':    NO Minimisation Possible'))
        else:
            data.append(guess)
            if i < 9:
                y_label_text = (' ' + chr(65+i) + ':' + y_label_variable_format)
            else:
                y_label_text = (' ' + chr(65+i) + ':' + y_label_variable_format)
            
            if maxiters[i] == 'yes':
                y_label_text += '+'

            if opti_var == 'bed_h':
                if not presentation:
                    y_labels.append(y_label_text.format(RMSE_opti_var[i],
                                                        DIFF_opti_var[i],
                                                        RMSE_sfc[i],
                                                        DIFF_sfc[i],
                                                        fct_opti_var[i],
                                                        times[i]))
                else:
                    y_labels.append(y_label_text.format(DIFF_opti_var[i],
                                                        fct_opti_var[i],
                                                        times[i]))
            elif opti_var in ['bed_shape', 'w0']:
                if not presentation:
                    y_labels.append(y_label_text.format(RMSE_opti_var[i],
                                                        DIFF_opti_var[i],
                                                        RMSE_w[i],
                                                        DIFF_w[i],
                                                        fct_opti_var[i],
                                                        times[i]))
                else:
                    y_labels.append(y_label_text.format(DIFF_opti_var[i],
                                                        DIFF_w[i],
                                                        fct_opti_var[i],
                                                        times[i]))
            else:
                raise ValueError('Unknown opti_var !')

    # make data an numpy array
    data = np.array(data)

    #choose colormap
    if not cmap_levels:
        color_nr = 100
        if opti_var == 'bed_h':
            cmap_limit = np.max(np.abs(check_first_guess))
            #cmap_limit = np.max(np.array([np.abs(np.floor(np.nanmin(np.array(data)))),
            #                              np.abs(np.ceil(np.nanmax(np.array(data))))]))
        elif opti_var in ['bed_shape', 'w0']:
            cmap_limit = np.max(np.abs(check_first_guess))
            #cmap_limit = np.max(np.array([np.abs(np.floor(np.nanmin(np.array(data)) * 10)),
            #                              np.abs(np.ceil(np.nanmax(np.array(data)) * 10))])) / 10
        else:
            raise ValueError('Unknown opti var!!')
        #if (np.min(data) < 0) & (np.max(data) > 0):
        cmap_levels = np.linspace(-cmap_limit, cmap_limit, color_nr, endpoint=True)
        #elif (np.min(data) < 0) & (np.max(data) =< 0):
        #    cmap_levels = np.linspace(-cmap_limit, 0, color_nr, endpoint=True)
        #elif (np.min(data) >= 0) & (np.max(data) > 0)
    else:
        color_nr = len(cmap_levels) - 1
    
    rel_color_steps = np.arange(color_nr)/color_nr
    if cmap == 'rainbow':
        colors = cm.rainbow(rel_color_steps)
    elif cmap == 'vlag':
        colors = sns.color_palette('vlag', color_nr)
    elif cmap == 'icefire':
        colors = sns.color_palette('icefire', color_nr)
    elif cmap == 'Spectral':
        colors = sns.color_palette('Spectral_r', color_nr)
    
    cmap = LinearSegmentedColormap.from_list('custom', colors, N=color_nr)
    cmap.set_bad(color='white')
    norm = mat_colors.BoundaryNorm(cmap_levels, cmap.N)

    # plot heatmap
    im = plt.imshow(data, aspect='auto', interpolation=None, cmap=cmap, norm=norm, alpha=1.)
    
    # Turn spines and ticks off and create white frame.
    for key, spine in ax.axis.items():
        spine.major_ticks.set_visible(False)
        spine.minor_ticks.set_visible(False)
        spine.line.set_visible(False)
        # spine.line.set_color(grid_color)
        # spine.line.set_linewidth(0) #grid_linewidth)
    
    # set y ticks
    ax.set_yticks(np.arange(data.shape[0]))
                
    ax.set_yticklabels(y_labels)
    #for tick in ax.get_yticklabels():
    #    tick.set_fontname("Arial")

    # align yticklabels left
    ax.axis["left"].major_ticklabels.set_ha("left")
    
    # set pad to put labels over heatmap
    ax.axis["left"].major_ticklabels.set_pad(labels_pad)

    # set y minor grid
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", axis='y', color=grid_color, linestyle='-', linewidth=grid_linewidth)
    
    # set x ticklabels off
    ax.set_xticklabels([])
    
    # create colorbar
    cax = ax.inset_axes([1.01, 0.1, 0.03, 0.8]) 
    #cax = fig.add_axes([0.5, 0, 0.01, 1])
    cbar = fig.colorbar(im, cax=cax, boundaries=cmap_levels, spacing='proportional',)
    cbar.set_ticks([np.min(cmap_levels),0,np.max(cmap_levels)])
    if opti_var == 'bed_h':
        cbar.set_ticklabels(['{:d}'.format(int(-cmap_limit)), '0' ,'{:d}'.format(int(cmap_limit))])
    elif opti_var == 'bed_shape':
        cbar.set_ticklabels(['{:.1f}'.format(-cmap_limit), '0' ,'{:.1f}'.format(cmap_limit)])
    elif opti_var == 'w0':
        cbar.set_ticklabels(['{:d}'.format(int(-cmap_limit)), '0' ,'{:d}'.format(int(cmap_limit))])
    else:
        raise ValueError('Unknown opti var!!')
    #cbar.ax.set_ylabel(cbarlabel,)
    
    # set title
    #ax.set_title(title)
    
    if annotation is not None:
        # include text
        ax.text(annotation_x_position, annotation_y_position, 
                 annotation,
                 horizontalalignment='left',
                 verticalalignment='center',
                 transform=ax.transAxes)
    
    return im

## legend plot

In [ ]:
def add_legend2(ax,
               title,
               fontsize,
               lw,
               ms,
               labels):
    
    ax.plot([],
            [],
            '-',
            lw=lw,
            ms=ms,
            c='none',
            label=labels[0])
    
    # plot for first gradient scaling
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c=color_1,
            label=labels[1])
    
    # plot for second gradient scaling
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c=color_2,
            zorder=5,
            label=labels[2])
    
    # plot for second gradient scaling
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c=color_3,
            zorder=5,
            label=labels[3])
    # plot for second gradient scaling
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c=color_4,
            zorder=5,
            label=labels[4])

    
    l = ax.legend(loc='center', fontsize=fontsize, title=title)
    plt.setp(l.get_title(), multialignment='center')
    ax.axis('off')

In [ ]:
def add_legend(ax,
               #title,
               fontsize,
               lw,
               ms,
               labels):
    
    ax.plot([],
            [],
            '-',
            lw=lw,
            ms=ms,
            c='none',
            label=labels[0])
    
    # plot for first gradient scaling
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c='none',
            label=labels[1])
    
    # plot for second gradient scaling
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c='none',
            zorder=5,
            label=labels[2])

    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c='none',
            zorder=5,
            label=labels[3])

    
    leg = ax.legend(loc='center',
                    fontsize=fontsize,
                    #title=title,
                    handlelength=0,
                    handletextpad=0,
                    fancybox=True)
    for item in leg.legendHandles:
        item.set_visible(False)
    ax.axis('off')

## performance plot

In [ ]:
def performance_plot(ax,
                     datasets,
                     fig=None,
                     # 'bed_h RMSE', 'bed_h Diff', 'bed_h Bias',
                     # 'bed_shape RMSE', 'bed_shape Diff', 'bed_shape Bias',
                     # 'w0 RMSE', 'w0 Diff', 'w0 Bias',
                     # 'sfc_h RMSE', 'sfc_h Diff', 'sfc_h Bias',
                     # 'widths RMSE', 'widths Diff', 'widths Bias'
                     performance_measurement='bed_h RMSE',
                     xlim=5,
                     y_label='',
                     annotation=None,
                     annotation_x_position=-0.2,
                     annotation_y_position=1,
                     lw=2,
                     fontsize=25,
                     ms=10,
                     nr_of_iterations=None,
                     ax_xlim=None
                    ):
    if not fig:
        fig = plt.gcf()

    measure = performance_measurement
    all_x = []
    all_y = []

    for dataset in datasets:
        if dataset is not None:
            max_index = len(dataset['computing_time'].values) - 1
            if nr_of_iterations is not None:
                max_index = nr_of_iterations - 1
            elif xlim is not None:
                # calculate all max diff surface_h
                all_DIFF_sfc_h = np.array(
                    [DIFF(dataset.true_surface_h.data,
                          dataset.surface_h.data[i-1])
                     for i in dataset.coords['nr_of_iteration'].data])
                # only consider as many points until max DIFF is smaller xlim
                if all_DIFF_sfc_h[-1] < xlim:
                    max_index = np.argmax(all_DIFF_sfc_h < xlim)

            # include time 0 for first guess
            tmp_x = [0]

            # add first guess values 
            if measure == 'bed_h RMSE':
                tmp_y = [RMSE(dataset['first_guessed_bed_h'], dataset['true_bed_h'])]
            elif measure == 'bed_h Diff':
                tmp_y = [DIFF(dataset['first_guessed_bed_h'], dataset['true_bed_h'])]
            elif measure == 'bed_h Bias':
                tmp_y = [BIAS(dataset['first_guessed_bed_h'], dataset['true_bed_h'])]

            elif measure == 'bed_shape RMSE':
                tmp_y = [RMSE(dataset['first_guessed_bed_shape'], dataset['true_bed_shape'])]
            elif measure == 'bed_shape Diff':
                tmp_y = [DIFF(dataset['first_guessed_bed_shape'], dataset['true_bed_shape'])]
            elif measure == 'bed_shape Bias':
                tmp_y = [BIAS(dataset['first_guessed_bed_shape'], dataset['true_bed_shape'])]

            elif measure == 'w0 RMSE':
                tmp_y = [RMSE(dataset['first_guessed_w0'], dataset['true_w0'])]
            elif measure == 'w0 Diff':
                tmp_y = [DIFF(dataset['first_guessed_w0'], dataset['true_w0'])]
            elif measure == 'w0 Bias':
                tmp_y = [BIAS(dataset['first_guessed_w0'], dataset['true_w0'])]

            elif measure == 'sfc_h RMSE':
                tmp_y = [RMSE(dataset['first_guess_surface_h'], dataset['true_surface_h'])]
            elif measure == 'sfc_h Diff':
                tmp_y = [DIFF(dataset['first_guess_surface_h'], dataset['true_surface_h'])]
            elif measure == 'sfc_h Bias':
                tmp_y = [DIFF(dataset['first_guess_surface_h'], dataset['true_surface_h'])]

            elif measure == 'widths RMSE':
                tmp_y = [RMSE(dataset['first_guess_widths'], dataset['true_widths'])]
            elif measure == 'widths Diff':
                tmp_y = [DIFF(dataset['first_guess_widths'], dataset['true_widths'])]
            elif measure == 'widths Bias':
                tmp_y = [DIFF(dataset['first_guess_widths'], dataset['true_widths'])]

            else:
                raise ValueError('Unknown performance measurement!')

            for i in dataset.coords['nr_of_iteration'].values[:max_index + 1] - 1:
                tmp_x.append(dataset['computing_time'][i])
                if measure == 'bed_h RMSE':
                    tmp_y.append(RMSE(dataset['guessed_bed_h'][i], dataset['true_bed_h']))
                elif measure == 'bed_h Diff':
                    tmp_y.append(DIFF(dataset['guessed_bed_h'][i], dataset['true_bed_h']))
                elif measure == 'bed_h Bias':
                    tmp_y.append(BIAS(dataset['guessed_bed_h'][i], dataset['true_bed_h']))

                elif measure == 'bed_shape RMSE':
                    tmp_y.append(RMSE(dataset['guessed_bed_shape'][i], dataset['true_bed_shape']))
                elif measure == 'bed_shape Diff':
                    tmp_y.append(DIFF(dataset['guessed_bed_shape'][i], dataset['true_bed_shape']))
                elif measure == 'bed_shape Bias':
                    tmp_y.append(BIAS(dataset['guessed_bed_shape'][i], dataset['true_bed_shape']))

                elif measure == 'w0 RMSE':
                    tmp_y.append(RMSE(dataset['guessed_w0'][i], dataset['true_w0']))
                elif measure == 'w0 Diff':
                    tmp_y.append(DIFF(dataset['guessed_w0'][i], dataset['true_w0']))
                elif measure == 'w0 Bias':
                    tmp_y.append(BIAS(dataset['guessed_w0'][i], dataset['true_w0']))

                elif measure == 'sfc_h RMSE':
                    tmp_y.append(RMSE(dataset['surface_h'][i], dataset['true_surface_h']))
                elif measure == 'sfc_h Diff':
                    tmp_y.append(DIFF(dataset['surface_h'][i], dataset['true_surface_h']))
                elif measure == 'sfc_h Bias':
                    tmp_y.append(BIAS(dataset['surface_h'][i], dataset['true_surface_h']))

                elif measure == 'widths RMSE':
                    tmp_y.append(RMSE(dataset['widths'][i], dataset['true_widths']))
                elif measure == 'widths Diff':
                    tmp_y.append(DIFF(dataset['widths'][i], dataset['true_widths']))
                elif measure == 'widths Bias':
                    tmp_y.append(BIAS(dataset['widths'][i], dataset['true_widths']))

                else:
                    raise ValueError('Unknown performance measurement!')
        else:
            tmp_x = []
            tmp_y = []

        all_x.append(tmp_x)
        all_y.append(tmp_y)

        colors = [color_1, color_2, color_3, color_4]
        for i, (x, y) in enumerate(zip(all_x, all_y)):
            ax.plot(x, y,
                    '.-',
                    lw=lw,
                    ms=ms,
                    c=colors[i])

        #ax.legend((),(),title=measure, loc='best')
        #ax.axvline(60, alpha=0.5, c='gray', ls='--')
        # ax.axvline(20, alpha=0.5, c='gray', ls='--')
        #if xlim is not None:
        #    ax.set_xlim(xlim)
    ax.tick_params(axis='both', colors=axis_color, width=lw)
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_color(axis_color)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.set_xlabel(r'$T_{cpu}$', fontsize=fontsize, c=axis_color)
    ax.set_ylabel(y_label, fontsize=fontsize, c=axis_color)
    
    if ax_xlim is not None:
        ax.set_xlim(ax_xlim)
    
    if annotation is not None:
        ax.text(annotation_x_position, annotation_y_position,
                annotation,
                horizontalalignment='left',
                verticalalignment='center',
                transform = ax.transAxes)

# Define Colors

In [ ]:
colors = sns.color_palette("colorblind")
colors

In [ ]:
axis_color = list(colors[7]) + [1.]
color_1 = list(colors[3]) + [1.]
color_2 = list(colors[0]) + [1.]
color_3 = list(colors[4]) + [1.]
color_4 = list(colors[2]) + [1.]
glacier_color = glacier_color = list(colors[9]) + [.5]

# Import Data

In [ ]:
input_folder = 'plot_data/'
filename_scale_1 = 'par_clif_cons_ret_bed_h_and_bed_shape_at_once_scal1reg11.nc'
filename_scale_1e4 = 'par_clif_cons_ret_bed_h_and_bed_shape_at_once_scal1e4reg11.nc'
filename_separated = 'par_clif_cons_ret_bed_h_and_bed_shape_separatedreg11.nc'
filename_calculated = 'par_clif_cons_ret_bed_h_and_bed_shape_calculatedreg11.nc'

datasets = []

for filename in [filename_scale_1, filename_separated, filename_scale_1e4, filename_calculated]:
    with xr.open_dataset(input_folder + filename) as ds:
        datasets.append(ds)

# Create figure with performance

In [ ]:
#dataset,
nr_of_iterations = None
facecolor = 'white'
labels_pad = -700
cmap = 'Spectral'
fontsize = 25
lw=2
ms=10
annotation_x_position_spatial = -0.15
annotation_y_position_spatial = 0.9
annotation_x_position_performance = -0.14
annotation_y_position_performance = 1.05
#index_start_first_profil_row = 0
#index_end_first_profil_row = 6
#index_start_second_profil_row = 65
#index_end_second_profil_row = 71
save_file = True
filename = 'par_methods_overview.pdf'

#plt.rcParams['font.family'] = 'monospace'
mpl.rcParams.update({'font.size': fontsize})

fig = plt.figure(figsize=(1,1), facecolor='white')

# define grid
total_width = 10
# define fixed size of spatial subplot
spatial_height = 2.5
spatial_y_separation = 0.5
# define fixed size for performance plot
performance_height = 2.5
performance_width = 8
performance_separation_y = 1
separation_y_performance_spatial = 0.5
# define fixed size for legend
legend_height = 3.5
separation_x_legend_spatial = 0.5

# fixed size in inch
# along x axis                                                            x-index for locator
horiz = [Size.Fixed(total_width),                                        # 0
         ]
                                                                          # y-index for locator
vert = [Size.Fixed(performance_height),                                   # 0 performance row 2
        Size.Fixed(separation_y_performance_spatial),
        Size.Fixed(performance_height),                                   # 2 performance row 1
        Size.Fixed(separation_y_performance_spatial),
        Size.Fixed(spatial_height),                                       # 4 spatial row 2
        Size.Fixed(spatial_y_separation),                                 
        Size.Fixed(spatial_height),                                       # 6 spatial row 1
        Size.Fixed(separation_x_legend_spatial),                                 
        Size.Fixed(legend_height),                                        # 8 legend
       ]

# define indices for subplots for easier changes later
# spatial heatmap
spatial_nx = 0
spatial_nx1 = 1
spatial_ny_row_1 = 6
spatial_ny_row_2 = 4
spatial_annotation = ['(a)', '(b)']
# performance
performance_nx = 0
performance_ny_row_1 = 2
performance_ny_row_2 = 0
# legend
legend_nx = 0
legend_ny = 8

# Position of the grid in the figure
rect = (0., 0., 1., 1.)  

# divide the axes rectangle into grid whose size is specified by horiz * vert
divider = Divider(fig, rect, horiz, vert, aspect=False)

with plt.rc_context({'font.family': 'monospace'}):
    ax = setup_axes(fig, 111)
    im = heatmap(datasets,
                 opti_var='bed_h',
                 annotation=spatial_annotation[0],
                 annotation_x_position=annotation_x_position_spatial,
                 annotation_y_position=annotation_y_position_spatial,
                 fig=fig,
                 ax=ax,
                 cmap=cmap,
                 grid_color=facecolor,
                 presentation=False,
                 labels_pad=labels_pad,
                 xlim=5,
                 nr_of_iterations=nr_of_iterations)
    ax.set_axes_locator(divider.new_locator(nx=spatial_nx,
                                            nx1=spatial_nx1,
                                            ny=spatial_ny_row_1))
    
    ax = setup_axes(fig, 111)
    im = heatmap(datasets,
                 opti_var='bed_shape',
                 annotation=spatial_annotation[1],
                 annotation_x_position=annotation_x_position_spatial,
                 annotation_y_position=annotation_y_position_spatial,
                 fig=fig,
                 ax=ax,
                 cmap='vlag',
                 grid_color=facecolor,
                 presentation=False,
                 labels_pad=labels_pad,
                 xlim=5,
                 nr_of_iterations=nr_of_iterations)
    ax.set_axes_locator(divider.new_locator(nx=spatial_nx,
                                            nx1=spatial_nx1,
                                            ny=spatial_ny_row_2))
# add perfomance plot bed_h RMSE
ax = fig.subplots()
performance_plot(ax,
                 datasets,
                 fig=None,
                 # 'bed_h RMSE', 'bed_h Diff', 'bed_h Bias',
                 # 'bed_shape RMSE', 'bed_shape Diff', 'bed_shape Bias',
                 # 'w0 RMSE', 'w0 Diff', 'w0 Bias',
                 # 'sfc_h RMSE', 'sfc_h Diff', 'sfc_h Bias',
                 # 'widths RMSE', 'widths Diff', 'widths Bias'
                 performance_measurement='bed_h RMSE',
                 xlim=5,
                 y_label='RMSE_b',
                 annotation='(c)',
                 annotation_x_position=annotation_x_position_performance,
                 annotation_y_position=annotation_y_position_performance,
                 lw=lw,
                 fontsize=fontsize,
                 ms=ms,
                 nr_of_iterations=nr_of_iterations,
                 ax_xlim=[0, 400]
                )
ax.set_axes_locator(divider.new_locator(nx=performance_nx,
                                        ny=performance_ny_row_1))

# add perfomance plot bed_shape RMSE
ax = fig.subplots()
performance_plot(ax,
                 datasets,
                 fig=None,
                 # 'bed_h RMSE', 'bed_h Diff', 'bed_h Bias',
                 # 'bed_shape RMSE', 'bed_shape Diff', 'bed_shape Bias',
                 # 'w0 RMSE', 'w0 Diff', 'w0 Bias',
                 # 'sfc_h RMSE', 'sfc_h Diff', 'sfc_h Bias',
                 # 'widths RMSE', 'widths Diff', 'widths Bias'
                 performance_measurement='bed_shape RMSE',
                 xlim=5,
                 y_label='RMSE_Ps',
                 annotation='(d)',
                 annotation_x_position=annotation_x_position_performance,
                 annotation_y_position=annotation_y_position_performance,
                 lw=lw,
                 fontsize=fontsize,
                 ms=ms,
                 nr_of_iterations=nr_of_iterations,
                 ax_xlim=[0, 350]
                )
ax.set_axes_locator(divider.new_locator(nx=performance_nx,
                                        ny=performance_ny_row_2))

# add legend
ax = fig.subplots()
add_legend2(ax=ax,
           title=(r'$\bf{cliff}$ with $\bf{constant}$ width and $\bf{parabolic}$ shape,' +
                   '\n' +
                   r'$\bf{retreating}$ from an $\bf{initial~ glacier~ surface}$,' +
                   '\n' + 
                   r'regularisation parameters $\lambda_0$ = 1 and $\lambda_1$ = 100'),
           fontsize=fontsize,
           lw=lw,
           ms=ms,
           labels=['fg: first guess',
                   "and A: 'explicit' without scaling",
                   "and B: 'iterative'",
                   "and C: 'explicit' with scaling of 1e-4",
                   "and D: 'implicit' with no limits"])
ax.set_axes_locator(divider.new_locator(nx=legend_nx,
                                        ny=legend_ny))

if save_file:
    fig.savefig(filename, format='pdf', bbox_inches='tight', dpi=300);